In [9]:
import pandas as pd 
import numpy as np

df  = pd.read_csv("newCsv.csv")
df.head()

,Document,Topic_group,Priority
0,I need to download my payslip for March,HR Support,Low
1,Where can I find my salary slip?,HR Support,Low
2,My laptop screen is broken and flickering,Hardware,Medium
3,I forgot my password and I am locked out,Access,High
4,We need to order 5 new monitors,Purchase,Low


In [10]:
df[df.Document == "Document"]

,Document,Topic_group,Priority


In [11]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 3283 entries, 0 to 3282
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Document     3283 non-null   str  
 1   Topic_group  3283 non-null   str  
 2   Priority     3283 non-null   str  
dtypes: str(3)
memory usage: 77.1 KB


In [13]:
import joblib
import os
import re
import pandas as pd

# ==========================================
# 1. SETUP PATHS
# ==========================================
# Finds the models folder automatically
current_dir = os.getcwd()
if "ml_training" in current_dir:
    # If running from inside ml_training, go up one level
    root_dir = os.path.dirname(current_dir)
else:
    # If running from root
    root_dir = current_dir

models_dir = os.path.join(root_dir, 'models')
dept_model_path = os.path.join(models_dir, 'ticket_pipeline.joblib')
prio_model_path = os.path.join(models_dir, 'priority_pipeline.joblib')

print(f"📂 Loading models from: {models_dir}")

# ==========================================
# 2. LOAD MODELS
# ==========================================
try:
    dept_model = joblib.load(dept_model_path)
    print("✅ Department Model Loaded")
except FileNotFoundError:
    print("❌ Error: Department model not found. Did you train it?")
    exit()

try:
    prio_model = joblib.load(prio_model_path)
    print("✅ Priority Model Loaded")
except FileNotFoundError:
    print("❌ Error: Priority model not found. Did you train it?")
    exit()

# ==========================================
# 3. CLEANING FUNCTION (MUST BE EXACT)
# ==========================================
def clean_text(text):
    text = str(text).lower()                 # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)     # Remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip() # Remove double spaces
    return text

# ==========================================
# 4. RUN DIAGNOSTIC TEST
# ==========================================
test_cases = [
    "I need to download my payslip for March",       # Exp: HR, Low
    "My laptop is on fire and smoking",              # Exp: Hardware, High
    "I cannot login to the VPN",                     # Exp: Access, High
    "We need to buy 5 new monitors",                 # Exp: Purchase, Low
    "My invoice is incorrect and double charged",    # Exp: Billing, Medium/High
]

print("\n" + "="*60)
print(f"{'INPUT TICKET':<45} | {'DEPT':<12} | {'PRIORITY'}")
print("="*60)

for ticket in test_cases:
    # 1. Clean
    clean_input = clean_text(ticket)
    
    # 2. Predict
    pred_dept = dept_model.predict([clean_input])[0]
    pred_prio = prio_model.predict([clean_input])[0]
    
    # 3. Print Row
    print(f"{ticket:<45} | {pred_dept:<12} | {pred_prio}")

print("="*60)

📂 Loading models from: c:\Users\Ankit Chaudhary\OneDrive\Desktop\ai-ticketing-system\models
✅ Department Model Loaded
✅ Priority Model Loaded

INPUT TICKET                                  | DEPT         | PRIORITY
I need to download my payslip for March       | HR Support   | Low
My laptop is on fire and smoking              | Hardware     | High
I cannot login to the VPN                     | Access       | High
We need to buy 5 new monitors                 | Purchase     | Low
My invoice is incorrect and double charged    | Billing      | High
